# IRS Finetune - Gemini-pro

## Setup

In [ ]:
# Setup directory
%cd E:/Github_Repo/Info-Retrieve-AI/

In [ ]:
# Install required packages
%pip install -r raga_requirements.txt
%pip install sentence_transformers rouge nltk bert-score

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import necessary modules
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup  # If web scraping is needed directly in RAGA
from huggingface_hub import HfApi  # For interacting with Hugging Face Hub if necessary
import ragas
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_relevancy, context_recall, context_entity_recall, answer_similarity, answer_correctness
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from gemini_script import QASystem, BlogIndexer # Use gemini_script.py
from __init__ import cfg
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from datasets import Dataset
from bert_score import score as bert_score


# Set environment variables for API access
os.environ['HF_HOME'] = '/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA/huggingface'
os.environ['TRANSFORMERS_CACHE'] = os.environ['HF_HOME']
os.environ['HF_DATASETS_CACHE'] = os.environ['HF_HOME']
os.environ['HF_METRICS_CACHE'] = os.environ['HF_HOME']
os.environ['HF_TOKEN'] = cfg.HF_TOKEN
os.environ['GOOGLE_API_KEY'] = cfg.GOOGLE_API_KEY
os.environ['OPENAI_API_KEY'] = cfg.OPENAI_API_KEY

nltk.download('punkt')
rouge = Rouge()

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.wh

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Synthetic Baseline Responses

In [ ]:
# Read the Dataset
data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA/syn_gpt_base_50qstns.csv")

# Fill NaN values in 'ground_truth' column with a specific placeholder
data['ground_truth'] = data['ground_truth'].fillna('No response available')
data

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What challenges do auto manufacturers face in ...,"['In both the UK and US, there is currently a ...",Severe price increases imposed via tariffs tha...,simple,[{'title': 'How Disruptive Pricing Will Impact...,True
1,What was the limitation of the quick-serve res...,[' quick-serve restaurant (QSR) we worked with...,The quick-serve restaurant's segmentation solu...,simple,"[{'title': 'Obtain a Deeper, Multidimensional ...",True
2,How does negative press coverage impact a bran...,[' spurred viral videos of fans destroying the...,No response available,simple,"[{'title': 'The Power, and Pitfalls, of Politi...",True
3,How does EVForward provide valuable insight in...,[' to much more effective results.\n\n\nEVForw...,EVForward provides valuable insight into the E...,simple,[{'title': 'Finding the Future of Mobility wit...,True
4,What challenges do landlords face when install...,"[' sharing (i.e., if multiple EVs are plugged ...",Landlords face a balancing act in installing i...,simple,[{'title': 'Multifamily Housing EV Charging: W...,True
5,How can effective sponsorship measurement help...,"[' marketers don’t see the value, or worse sti...",Effective sponsorship measurement provides det...,simple,[{'title': 'Measuring the Value of Sponsorship...,True
6,What is the current availability of charging s...,['EV Potential Not Yet Realized\n\n\nAs we loo...,The number of public charging stations has inc...,simple,"[{'title': 'The Challenges of Making EV Easy',...",True
7,How is the global rideshare industry projected...,['It’s safe to say that ridesharing services l...,The global rideshare industry is projected to ...,simple,[{'title': 'Rideshare Vehicles: A Meaningful B...,True
8,What is the importance of delivering multiface...,['there’s an obstacle in my way\n—\nperhaps I ...,The critical support offered to EV buyers need...,simple,[{'title': 'How to Create an “EV Easy” Mindset...,True
9,What are some of the specific areas that manuf...,"['For fans of the Terminator series of movies,...","Manufacturers, suppliers, and technology compa...",simple,[{'title': 'Rise of the Smart Car: The Shape-S...,True


## LLM Performance Evaluator

In [ ]:
class LLMPerformanceEvaluator:
    def __init__(self, model, indexer, data):
        self.model = model
        self.indexer = indexer
        self.data = data

    def generate_responses(self):
        # Generate responses for each question using the model
        self.data['Response'] = self.data['question'].apply(lambda x: self.model.answer_query(x))

    def compute_metrics(self):
        # Ensure ground truth data is present for all records to avoid processing errors
        filtered_data = self.data.dropna(subset=['ground_truth'])

        # Preprocessing text for metric calculation
        filtered_data['Reference_Tokens'] = filtered_data['ground_truth'].apply(lambda x: nltk.word_tokenize(x.lower()))
        filtered_data['Response_Tokens'] = filtered_data['Response'].apply(lambda x: nltk.word_tokenize(x.lower()))

        # Calculate BLEU and ROUGE scores only for entries with valid ground truth
        filtered_data['BLEU'] = filtered_data.apply(lambda row: sentence_bleu([row['Reference_Tokens']], row['Response_Tokens'], weights=(0.25, 0.25, 0.25, 0.25)), axis=1)
        filtered_data['ROGUE'] = filtered_data.apply(lambda row: rouge.get_scores(row['Response'], row['ground_truth'])[0]['rouge-l']['f'], axis=1)

        # Calculate BERT scores
        # Convert to list for BERT score calculation
        references = filtered_data['ground_truth'].tolist()
        predictions = filtered_data['Response'].tolist()

        # Calculate BERT scores
        P, R, F1 = bert_score(predictions, references, lang="en", verbose=True)

        # Add BERT scores to DataFrame
        filtered_data['BERT_P'] = P.numpy()
        filtered_data['BERT_R'] = R.numpy()
        filtered_data['BERT_F1'] = F1.numpy()

        # filtered_data['BERT'] = bert_score(filtered_data['Response'], filtered_data['ground_truth'], lang="en")[2]

        # Prepare data samples for RAGA metrics, ensuring all necessary fields are present
        data_samples = {
            'question': filtered_data['question'].tolist(),
            'answer': filtered_data['Response'].tolist(),
            'contexts': [nltk.sent_tokenize(x) for x in filtered_data['ground_truth']],
            'ground_truth': filtered_data['ground_truth'].tolist()  # Ensuring ground_truth is included for RAGA metrics
        }
        dataset = Dataset.from_dict(data_samples)

        # Compute RAGA metrics
        raga_results = evaluate(dataset, metrics=[
            faithfulness, answer_relevancy, context_precision, context_relevancy,
            context_recall, context_entity_recall, answer_similarity, answer_correctness
        ])
        raga_df = raga_results.to_pandas()

        # Combine all metrics into a single DataFrame
        self.metrics_df = pd.concat([filtered_data, raga_df], axis=1)
        return self.metrics_df


## Generate Responses : Gemini-pro

In [ ]:
# Generate response
indexer = BlogIndexer(
    url='https://escalent.co/thought-leadership/blog/?industry=automotive-and-mobility',
    headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}
)
model = QASystem('gemini-pro', indexer)

evaluator = LLMPerformanceEvaluator(model, indexer, data)
evaluator.generate_responses()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response based on the context...
Response generated.
Generating query embedding...
Retrieving context...
Generating response 

## Compute Performance Metrics

In [ ]:
evaluator.compute_metrics()
evaluator.metrics_df.to_csv(r"E:\Github_Repo\Info-Retrieve-AI\RAGAs\output\gemini_performance_metrics.csv", index=False)
print("Performance metrics have been saved successfully.")
# print(evaluator.metrics_df.head())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 441.42 seconds, 0.11 sentences/sec


Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

Performance metrics have been saved successfully.


In [ ]:
# Set the pandas options to display more columns
pd.set_option('display.max_columns', None)
evaluator.metrics_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,Response,Reference_Tokens,Response_Tokens,BLEU,...,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_relevancy,context_recall,context_entity_recall,answer_similarity,answer_correctness
0,What challenges do auto manufacturers face in ...,"['In both the UK and US, there is currently a ...",Severe price increases imposed via tariffs tha...,simple,[{'title': 'How Disruptive Pricing Will Impact...,True,**Challenges for Auto Manufacturers Due to Pot...,"[severe, price, increases, imposed, via, tarif...","[*, *, challenges, for, auto, manufacturers, d...",3.008812e-02,...,[Severe price increases imposed via tariffs th...,Severe price increases imposed via tariffs tha...,1.0,0.990047,1.0,0.333333,1.0,0.909091,0.875285,0.414474
1,What was the limitation of the quick-serve res...,[' quick-serve restaurant (QSR) we worked with...,The quick-serve restaurant's segmentation solu...,simple,"[{'title': 'Obtain a Deeper, Multidimensional ...",True,The quick-serve restaurant's segmentation solu...,"[the, quick-serve, restaurant, 's, segmentatio...","[the, quick-serve, restaurant, 's, segmentatio...",3.171809e-01,...,[The quick-serve restaurant's segmentation sol...,The quick-serve restaurant's segmentation solu...,1.0,0.994279,1.0,1.000000,1.0,1.000000,0.967174,0.670366
2,How does negative press coverage impact a bran...,[' spurred viral videos of fans destroying the...,No response available,simple,"[{'title': 'The Power, and Pitfalls, of Politi...",True,The information provided does not directly add...,"[no, response, available]","[the, information, provided, does, not, direct...",0.000000e+00,...,[No response available],No response available,NaN,0.000000,0.0,0.000000,0.0,1.000000,0.792917,0.198205
3,How does EVForward provide valuable insight in...,[' to much more effective results.\n\n\nEVForw...,EVForward provides valuable insight into the E...,simple,[{'title': 'Finding the Future of Mobility wit...,True,EVForward provides valuable insight into the E...,"[evforward, provides, valuable, insight, into,...","[evforward, provides, valuable, insight, into,...",2.336828e-01,...,[EVForward provides valuable insight into the ...,EVForward provides valuable insight into the E...,0.4,0.894055,1.0,1.000000,1.0,1.000000,0.954006,0.738501
4,What challenges do landlords face when install...,"[' sharing (i.e., if multiple EVs are plugged ...",Landlords face a balancing act in installing i...,simple,[{'title': 'Multifamily Housing EV Charging: W...,True,Landlords face several challenges when install...,"[landlords, face, a, balancing, act, in, insta...","[landlords, face, several, challenges, when, i...",7.335263e-79,...,[Landlords face a balancing act in installing ...,Landlords face a balancing act in installing i...,1.0,0.998204,1.0,0.666667,1.0,0.833333,0.944082,0.902686


## Test

In [ ]:
print(evaluator.metrics_df[['BERT_P', 'BERT_R', 'BERT_F1']].head())

     BERT_P    BERT_R   BERT_F1
0  0.813526  0.882351  0.846542
1  0.923757  0.968018  0.945370
2  0.798475  0.834026  0.815863
3  0.873407  0.942777  0.906767
4  0.847477  0.869530  0.858362
